In [3]:
from skimage.feature import greycomatrix
import skimage
import cv2
import numpy as np

# Calculate the GLCM of an image
image=cv2.imread("/Users/taiaburrahman/Desktop/git/RetinaSegmentation-HEx/Results/HardExodus/Tests/IDRiD_55.jpg")
distances = [1, 2, 3]
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
glcm = greycomatrix(image, distances=distances, angles=angles, symmetric=True, normed=True)
features = {prop: skimage.feature.greycoprops(glcm, prop)[0, 0] for prop in properties}


/Users/taiaburrahman/Desktop/git/RetinaSegmentation-HEx/venv/lib/python3.9/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')


ValueError: The parameter `image` must be a 2-dimensional array